In [ ]:
%pip install --upgrade --user google-cloud-aiplatform>=1.29.0 google-cloud-storage

In [3]:
import os

In [6]:
%pwd

'd:\\Cite-What-You-Type'

In [5]:
os.chdir("../")

In [20]:
from google.cloud import aiplatform
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
import os
import json
from typing import List
from google.cloud import storage
import uuid
from langchain_google_vertexai import VectorSearchVectorStore
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever 
from sklearn.feature_extraction.text import TfidfVectorizer
from langchain_core.documents import Document
from Multi_Modal.chunking import get_chunks
from Multi_Modal.SeperationAndSummarization import summarize_chunks
from conversion import convert_to_pdf
from pinecone import Pinecone,ServerlessSpec
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
load_dotenv()

True

In [9]:
PROJECT_ID = os.environ['PROJECT_ID']
REGION = os.environ['REGION']
BUCKET_NAME=os.environ['BUCKET_NAME']
INDEX_DISPLAY_NAME=os.environ['INDEX_DISPLAY_NAME']

In [10]:
aiplatform.init(project=PROJECT_ID,location=REGION)

embeddings= GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [ ]:
def upload_vector_to_gcs(documents, bucket_name=BUCKET_NAME):

    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    print("Completed Login")

    texts = []
    valid_indices = []
    
    for i, doc in enumerate(documents):
        try:
            raw_text = ""
            if hasattr(doc, "metadata") and "original_content" in doc.metadata:
                content_str = doc.metadata.get("original_content")
                if content_str:
                    data = json.loads(content_str)
                    raw_text = data.get('raw_text', "")
            
            if not raw_text:
                raw_text = doc.page_content 

            texts.append(raw_text)
            valid_indices.append(i)
            
        except Exception as e:
            print(f"Error parsing doc {i}: {e}")
            texts.append("") 

    print(f"Generating embeddings for {len(texts)} documents...")
    vectors = embeddings.embed_documents(texts)
    if vectors:
        print(f"!!! ACTUAL VECTOR DIMENSION: {len(vectors[0])} !!!") 


    vertex_content = []
    
    for i, doc_index in enumerate(valid_indices):
        doc = documents[doc_index]
        
        docs_id = str(uuid.uuid4()) 
        
        metadata_blob = bucket.blob(f"docstore/{docs_id}.json")
        
        try:
            content = json.loads(doc.metadata.get("original_content", "{}"))
        except:
            content = {}

        bucket_content = {
            'id': docs_id,
            'raw_text': content.get('raw_text', texts[i]), 
            'table_as_html': content.get('table_html', []),
            'image_base64': content.get('image_base64', [])
        }
        
        metadata_blob.upload_from_string(json.dumps(bucket_content))

        vertex_record = {
            "id": docs_id,
            "embedding": vectors[i] 
        }
        vertex_content.append(json.dumps(vertex_record))

    vector_data = "\n".join(vertex_content)
    
    unique_folder = f"init_vectors_{uuid.uuid4()}"
    
    blob_name = f"{unique_folder}/vectors.json"
    vector_blob = bucket.blob(blob_name)
    vector_blob.upload_from_string(vector_data)

    print(f"Success! Metadata in gs://{bucket_name}/docstore/")
    print(f"Vectors ready in gs://{bucket_name}/{unique_folder}/")
    
    gcs_uri = f"gs://{bucket_name}/{unique_folder}/"
    return gcs_uri


In [ ]:
hasattr(docs[0],"page_content")

In [ ]:
docs[0].metadata["original_content"]

In [21]:
def upload_files(documents: List[Document], index_name: str, bucket_name: str = BUCKET_NAME):


    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    pc = Pinecone()

    print("Login successful to GCP & Pinecone")

    texts = []
    for i, doc in enumerate(documents):
        try:
            if hasattr(doc, "page_content") and doc.page_content:
                texts.append(doc.page_content)
            else:
                texts.append("")  
        except Exception as e:
            print(f"Error while parsing document {i}, error:", e)
            raise  

    print(f"Generating embeddings for {len(texts)} documents...")
    vectors = embeddings.embed_documents(texts)
    vectorizer = TfidfVectorizer()
    tfdif_matrix=vectorizer.fit_transform(texts)
    
    if not vectors:
        raise ValueError("No embeddings generated; check your documents/embeddings model.")

    print(f"!!! ACTUAL VECTOR DIMENSION: {len(vectors[0])} !!!")
    dimension = len(vectors[0])

    print("Initializing index in Pinecone")
    existing_indexes = [i["name"] for i in pc.list_indexes()]
    if index_name not in existing_indexes:
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine",
            spec=ServerlessSpec(cloud="gcp", region="us-central1")
        )
        print(f"Created Pinecone index: {index_name}")
    else:
        print(f"Pinecone index '{index_name}' already exists")


    pinecone_docs: List[Document] = []

    for i, doc in enumerate(documents):
        doc_id = uuid.uuid4().hex[:15]

        try:
            original_content = doc.metadata.get("original_content", "{}")
            content = json.loads(original_content) if isinstance(original_content, str) else original_content
        except Exception:
            content = {}

        bucket_content = {
            "id": doc_id,
            "raw_text": content.get("raw_text", texts[i]),
            "table_as_html": content.get("table_html", []),
            "image_base64": content.get("image_base64", []),
        }

        metadata_blob = bucket.blob(f"docstore/{doc_id}.json")
        metadata_blob.upload_from_string(json.dumps(bucket_content))

        pinecone_doc = Document(
            page_content=texts[i],
            metadata={
                "id": doc_id,
                "gcs_uri": f"gs://{bucket_name}/docstore/{doc_id}.json",
                "source": doc.metadata.get("source", "unknown"),
            },
        )
        pinecone_docs.append(pinecone_doc)

    print("Upserting documents into Pinecone via LangChain...")
    vector_store = PineconeVectorStore.from_documents(
        documents=pinecone_docs,
        embedding=embeddings,
        index_name=index_name,
    )

    print("Ingestion complete: GCS + Pinecone hybrid store is ready.")
    return tfdif_matrix,vector_store


In [ ]:
def create_and_deploy_index(gcs_uri):

    import traceback
    print("Creating Index (takes time)...")

    dimensions = 3072
    approximate_neighbors_count = 150
    leaf_node_embedding_count = 100
    leaf_nodes_to_search_percent = 10
    distance_measure_type = "DOT_PRODUCT_DISTANCE"

    print("Index params:", {
        'dimensions': dimensions,
        'approximate_neighbors_count': approximate_neighbors_count,
        'leaf_node_embedding_count': leaf_node_embedding_count,
        'leaf_nodes_to_search_percent': leaf_nodes_to_search_percent,
        'distance_measure_type': distance_measure_type,
        'contents_delta_uri': gcs_uri,
    })

    try:
        my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
            display_name=INDEX_DISPLAY_NAME,
            contents_delta_uri=gcs_uri,
            dimensions=dimensions,
            approximate_neighbors_count=approximate_neighbors_count,
            leaf_node_embedding_count=leaf_node_embedding_count,
            leaf_nodes_to_search_percent=leaf_nodes_to_search_percent,
            distance_measure_type=distance_measure_type,
        )
    except Exception as e:
        print("Index creation failed. Exception repr:", repr(e))
        try:
            print("Exception type:", type(e))
            if hasattr(e, 'errors'):
                print("e.errors:", e.errors)
        except Exception as diag_exc:
            print("Error printing exception attributes:", diag_exc)
        print("Full traceback:")
        print(traceback.format_exc())
        raise

    print("Creating Endpoint")
    try:
        my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
            display_name=f"{INDEX_DISPLAY_NAME}_endpoint",
            public_endpoint_enabled=True
        )
    except Exception as e:
        print("Endpoint creation failed:", repr(e))
        print(traceback.format_exc())
        raise

    print("Deploying Index to Endpoint")
    try:
        my_index_endpoint.deploy_index(
            index=my_index,
            deployed_index_id="soumya_deployed_v1",
        )
    except Exception as e:
        print("Deploy failed:", repr(e))
        print(traceback.format_exc())
        raise

    print("deployment Completed")
    return my_index_endpoint


In [ ]:
def search_vertex_ai(query,index_endpoint, deployed_index_id, bucket_name=BUCKET_NAME):

    query_emb = embeddings.embed_query(query)

    response = index_endpoint.find_neighbors(
        deployed_index_id=deployed_index_id,
        queries = [query_emb],
        num_neighbors=5
    )

    print("Response:",response)


    results=[]
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    print("Login successed")

    for neighbor in response[0]:
        doc_id = neighbor.id
        score = neighbor.distance

        blob = bucket.blob(f"docstore/{doc_id}.json")
        if blob.exists():
            data = json.loads(blob.download_as_text())

            doc = Document(
                page_content=data.get('raw_text', ""), 
                metadata={
                    "tables": data.get('table_as_html', []),
                    "images": data.get('image_base64', []), 
                }
            )


        results.append(doc)
        
    return results


In [13]:
file_path = "pdfs/22-25 Clustering, K-means, DBSCAN.pdf"
output_dir = "temp_uploads"

In [12]:
from pathlib import Path

In [13]:
output_dir= Path(output_dir)
output_dir.mkdir(exist_ok=True)

In [15]:
query1= "Explain the workflow of the project. explain the tech stack used to build the project and tell how the query is procceed when a query is given to the final model"

In [15]:
print("Creating Chunks")
chunks = get_chunks(file_path)
print("Chunks Created")

print("summarize_chunks")
docs = summarize_chunks(chunks)
print("Summarization Completed")

Creating Chunks


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Total elements extracted ->  352
Chunks created:  25
Chunks Created
summarize_chunks
...Processing Chunk ...
Processed Chunk 1/25
Types Found:  ['text', 'Image']
Tables: 0 , Image: 1
Creating Summary...
Successfully Summarized
Preview:**1. Key facts, exact numbers, and data points**
*   **Document Title:** Clustering: Introduction, K...
Processed Chunk 2/25
Types Found:  ['text', 'Image']
Tables: 0 , Image: 1
Creating Summary...
Successfully Summarized
Preview:**1. Key facts, exact numbers, and data points:**
*   **Document Title:** "Clustering techniques"
* ...
Processed Chunk 3/25
Types Found:  ['Table', 'text', 'Image']
Tables: 1 , Image: 1
Creating Summary...
Successfully Summarized
Preview:### SEARCHABLE DESCRIPTION

**1. Key Facts, Exact Numbers, and Data Points**
*   **Definition:** Clu...
Processed Chunk 4/25
Types Found:  ['text', 'Image']
Tables: 0 , Image: 1
Creating Summary...
Successfully Summarized
Preview:**SEARCHABLE DESCRIPTION:**

**1. Key facts, exact numbers, and da

In [18]:
docs[0].page_content

'**1. Key facts, exact numbers, and data points**\n*   **Document Title:** Clustering: Introduction, K-Mean, and DBSCAN\n*   **Author:** Puneet Kumar Jain, PhD\n*   **Author\'s Affiliation:** Assistant Professor, Computer Science and Engineering (CSE) Department, National Institute of Technology Rourkela (NIT Rourkela).\n*   **Referenced Resources & Authors:**\n    *   **Book:** "Introduction to Data Mining" by Pang-Ning Tan (Michigan State University), Michael Steinbach (University of Minnesota), and Vipin Kumar (University of Minnesota). Resource URL: http://www-users.cs.umn.edu/~kumar/dmbook/index.php\n    *   **Presentation 1:** http://www.cse.ust.hk/~qyang/337/slides/cluster.ppt (Hong Kong University of Science and Technology)\n    *   **Presentation 2:** http://www2.cs.uh.edu/~ceick/ML/Topic9.ppt (University of Houston)\n    *   **Authors/Researchers:** www.cs.uiuc.edu/~hanj (Jiawei Han, University of Illinois Urbana-Champaign) and Martin Pfeifle (www.dbs.informatik.uni-muenchen.

In [ ]:
docs[1].metadata.get('original_content')

In [22]:
vectorizer,vector_store=upload_files(docs,"hybrid-search",BUCKET_NAME)

Login successful to GCP & Pinecone
Generating embeddings for 25 documents...
!!! ACTUAL VECTOR DIMENSION: 3072 !!!
Initializing index in Pinecone
Pinecone index 'hybrid-search' already exists
Upserting documents into Pinecone via LangChain...
Ingestion complete: GCS + Pinecone hybrid store is ready.


In [ ]:
pc=Pinecone()
index = pc.Index(host="https://soumya-index-fughegr.svc.gcp-us-central1-4a9f.pinecone.io")

In [ ]:
def hybrid_pinecone_retriever(query,vectorizer,index_name,top_k=5,bucket_name=BUCKET_NAME):

    pc=Pinecone()
    index = pc.Index(index_name)
    client = storage.Client(project=PROJECT_ID)
    bucket=client.bucket(bucket_name)

    print("Login successful to GCP & Pinecone")

    emb_query = embeddings.embed_query(query)
    query_vec=vectorizer.transform(query)
    sim=cosine_similarity(query_vec,vectorizer)[0]

    res= index.query(
        vector=emb_query,
        top_k=top_k,
        include_metadata=True
    )
    docs=[]
    try:
        for result in res['matches']:
            try:
                if "metadata" in result:
                    metadata = result.get("metadata","{}")
                    if metadata:
                        id=metadata.get("id","")
                        score= result["score"]
                        gcs_uri=metadata.get('gcs_uri',"")

                        if not gcs_uri:
                            continue
                    
                        path=gcs_uri.replace("gs://","")
                        bucket_name_gcs,*blob_parts= path.split("/")
                        blob_name ="/".join(blob_parts)

                        print(path)
                        print(bucket_name," ",blob_parts)

                        bucket = client.bucket(bucket_name_gcs)
                        blob = bucket.blob(blob_name)

                        if not blob.exists():
                            continue

                        data = json.loads(blob.download_as_text())

                        raw_text = data.get("raw_text", "")
                        tables   = data.get("table_as_html", [])
                        images   = data.get("image_base64", [])

                        doc = Document(
                            page_content=raw_text,
                            metadata={
                                "id": id,
                                "score": score,
                                "tables": tables,
                                "images": images,
                                "gcs_uri": gcs_uri,
                            }
                        )

                        docs.append(doc)

            except Exception as e:
                print("Error , Unable to fetch metadata",e)

        return docs
    except Exception as e:
        print(f"Error:{e}")

In [ ]:
document=hybrid_pinecone_retriever(query1,"soumya-index")

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k":10})

In [ ]:
document

In [ ]:
print("upload to GCS")
gcs_uri= upload_vector_to_gcs(docs)
print("Uploaded Completed")



In [ ]:
index_endpoint= create_and_deploy_index(gcs_uri)
print('Completed deploying index')

In [ ]:
endpoints = aiplatform.MatchingEngineIndexEndpoint.list()
for ep in endpoints:
    print(f"Name: {ep.display_name}")
    print(f"ID: {ep.name}")  


In [ ]:
my_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    index_endpoint_name=f"projects/{os.environ['PROJECT_NO']}/locations/{REGION}/indexEndpoints/{os.environ['ENDPOINT_ID']}"
)


In [ ]:
results = search_vertex_ai(
    query=query, 
    index_endpoint=my_endpoint,      
    deployed_index_id="soumya_deployed_v1", 
    bucket_name=BUCKET_NAME
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import base64


In [ ]:
results[0].metadata

In [ ]:
hasattr(results[0],"page_content")

In [ ]:
document

In [ ]:
def gen_final_ans(results,query):
    try:
        llm = ChatGoogleGenerativeAI(model='gemini-2.5-pro', temperature=0.1)

        prompt_text = f"""
        Based on the following document context, please answer this question: {query}
        
        CONTENT_TO_ANALYZE:
        """
        all_images_base64 = []
        for i, chunk in enumerate(results):
            prompt_text += f"\n--- Document Fragment {i+1} ---\n"
            if hasattr(chunk,"page_content"):
                raw_text = chunk.page_content
                if raw_text:
                    prompt_text += f"Text:\n{raw_text}\n\n"
            
            if hasattr(chunk,"metdata"):
                table= chunk.metadata.get("tables",[])
                if table:
                    for j , cnt in enumerate(table):
                        prompt_text += f"Table {j+1}:\n{cnt}\n\n"
                imgs = chunk.metadata.get("images",[])
                if imgs:
                    all_images_base64.extend(imgs)
            
        prompt_text += """ 
            INSTRUCTIONS:
            Provide a clear, comprehensive answer using the text, tables, and images provided above. 
            If the documents don't contain sufficient information to answer the question, state: "I don't have enough information to answer the question."
            
            ANSWER:"""
        message_content = [{'type': 'text', 'text': prompt_text}]

        for img_b64 in all_images_base64:
            message_content.append({"type": "image_url","image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}})

        message = HumanMessage(content=message_content)
        response = llm.invoke([message])

        return response.content
        
    except Exception as e:
        print(f"Answer gen failed: {e}")
        return 'Sorry, I encountered an error generating the answer.'

                


            


In [ ]:
ans=gen_final_ans(document,query1) 

In [ ]:
print(ans)